In [9]:
import pandas as pd
import numpy as np
import os, sys, gc, random
import datetime
from dateutil.relativedelta import relativedelta
from datetime import datetime
from glob import glob

# Machine learning
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer # 누락값 대체 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [10]:
class CFG:
    TOTAL_THRES = 300 # 구매액 임계값
    SEED = 42 # 랜덤 시드
    TARGET_YEAR_MONTH = '2011-12' # 예측할 연월 설정

cfg=CFG()
data_dir='/opt/ml/code/input/'
submit=pd.read_csv(data_dir+'sample_submission.csv')
data=pd.read_csv(data_dir+'train.csv',parse_dates=['order_date']) # parse_dates:datetime 형태로 변경되어야 하는 컬럼 지정

In [11]:
############SET SEED##############
# 시드 고정 함수
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(cfg.SEED)

In [12]:
############BEFORE FEATURE ENGINEERING##############
def generate_label(data,tgt_ym,total_thres=cfg.TOTAL_THRES,print_log=False): # tgt_ym는 test때는 TARGET_YEAR_MONTH. 아닐때는 임의로 넣어 train
    '''
    입력인자로 받는 tgt_ym에 대해 고객 ID별로 총 구매액이
    구매액 임계값을 넘는지 여부의 binary label을 생성하는 함수
    '''
    df=data.copy()
    df['ym']=df['order_date'].dt.strftime('%Y-%m')
    df.reset_index(drop=True, inplace=True)
    
    # 입력 연월 이전 고객 아이디 - 다음 월의 소비금액 예측 위험
    customer_id=df[df['ym']<tgt_ym]['customer_id'].unique()
    # 예측할 연월 데이터 선택
    df=df[df['ym']==tgt_ym]
    
    # label dataframe
    label=pd.DataFrame({'customer_id':customer_id})
    label['ym']=tgt_ym
    
    # 특정 연월에 해당하는 고객 id의 구매액 합산
    grped=df.groupby(['customer_id','ym'], as_index=False)['total'].sum()
    
    # merge grped result
    label=label.merge(grped,on=['customer_id','ym'],how='left')
    label.fillna(0.0,inplace=True)
    label['label']=(label['total']>total_thres).astype(int)
    
    # 정리
    label = label.sort_values('customer_id').reset_index(drop=True)
    if print_log:
        print(f'{tgt_ym} - final label shape: {label.shape}')
    
    return label
    
# generate_label(data,'2011-09',cfg.TOTAL_THRES)

In [13]:
############BEFORE FEATURE ENGINEERING##############
# 범주형->수치형 변환, 결측치 처리 
def feature_preprocessing(train, test, features, do_imputing=True):
    x_tr = train.copy()
    x_te = test.copy()
    
    # 범주형 피처 이름을 저장할 변수
    cate_cols = []

    # 레이블 인코딩
    for f in features:
        if x_tr[f].dtype.name == 'object': # 데이터 타입이 object(str)이면 레이블 인코딩
            cate_cols.append(f)
            le = LabelEncoder()
            # train + test 데이터를 합쳐서 레이블 인코딩 함수에 fit
            le.fit(list(x_tr[f].values) + list(x_te[f].values))
            
            # train 데이터 레이블 인코딩 변환 수행
            x_tr[f] = le.transform(list(x_tr[f].values))
            print(x_tr[f])
            
            # test 데이터 레이블 인코딩 변환 수행
            x_te[f] = le.transform(list(x_te[f].values))

    print('categorical feature:', cate_cols) # 범주형이 없었으면 [] 들어있는게 맞는 거겠지?

    if do_imputing:
        # 중위값으로 결측치 채우기
        imputer = SimpleImputer(strategy='median')

        x_tr[features] = imputer.fit_transform(x_tr[features])
        x_te[features] = imputer.transform(x_te[features])
    
    return x_tr, x_te

In [14]:
############FEATURE ENGINEERING##############
def feature_engineering1(data,tgt_ym):
    df=data.copy()

    # tgt_ym 이전 달 계산. tgt_ym이 실제 예측할 연월일 수도 있고, 임의일 수도 있음(train때는 임의 연월, 그 임의 연월의 전달로 설정됨)
    d=datetime.strptime(tgt_ym,'%Y-%m') # strptime : 문자열->datetime
    prev_ym=(d-relativedelta(months=1)).strftime('%Y-%m') # strftime : datetime->문자열

    # train, test 데이터 선택
    train=df[df['order_date']<prev_ym]
    test=df[df['order_date']<tgt_ym]
    
    # train, test 레이블 데이터 생성 - total값은 필요없어서 선택 안함
    train_label = generate_label(df, prev_ym)[['customer_id','ym','label']]
    test_label = generate_label(df, tgt_ym)[['customer_id','ym','label']]
    
    # group by aggregation 함수 선언
    agg_func = ['mean','max','min','sum','count','std','skew']
    all_train_data = pd.DataFrame()
    for i,tr_ym in enumerate(train_label['ym'].unique()): # 이거 왜 굳이 for문씀? 딱 한번만 돌아가는데 ...
        train_agg=train[train['order_date']<tr_ym].groupby(['customer_id']).agg(agg_func)
#         print(train_agg.columns.levels) # [['quantity', 'price', 'total'], ['mean', 'max', 'min', 'sum', 'count', 'std', 'skew']]
        
        # 멀티 레벨 컬럼을 사용하기 쉽게 1 레벨 컬럼명으로 변경
        new_cols = []
        for col in train_agg.columns.levels[0]:
            for stat in train_agg.columns.levels[1]:
                new_cols.append(f'{col}-{stat}')
        train_agg.columns=new_cols
        train_agg.reset_index(inplace = True)
        
        train_agg['ym']=tr_ym
        
        all_train_data = all_train_data.append(train_agg)
        
    all_train_data = train_label.merge(all_train_data, on=['customer_id', 'ym'], how='left') # feature 반영 데이터프레임
    features = all_train_data.drop(columns=['customer_id', 'label', 'ym']).columns # feature (agg정보)
    
    # group by aggretation 함수로 test 데이터 피처 생성
    test_agg = test.groupby(['customer_id']).agg(agg_func)
    test_agg.columns = new_cols
    
    test_data = test_label.merge(test_agg, on=['customer_id'], how='left')

    # train, test 데이터 전처리
    x_tr, x_te = feature_preprocessing(all_train_data, test_data, features)
    
#     print('x_tr.shape', x_tr.shape, ', x_te.shape', x_te.shape)
    
    return x_tr, x_te, all_train_data['label'], features

x_tr,x_te,all_data,features=feature_engineering1(data,'2011-09')

categorical feature: []


In [15]:
features

Index(['quantity-mean', 'quantity-max', 'quantity-min', 'quantity-sum',
       'quantity-count', 'quantity-std', 'quantity-skew', 'price-mean',
       'price-max', 'price-min', 'price-sum', 'price-count', 'price-std',
       'price-skew', 'total-mean', 'total-max', 'total-min', 'total-sum',
       'total-count', 'total-std', 'total-skew'],
      dtype='object')

In [16]:
x_tr

,customer_id,ym,label,quantity-mean,quantity-max,quantity-min,quantity-sum,quantity-count,quantity-std,quantity-skew,...,price-count,price-std,price-skew,total-mean,total-max,total-min,total-sum,total-count,total-std,total-skew
0,12346,2011-08,0,1.127660,74215.0,-74215.0,53.0,47.0,15474.897433,-0.000228,...,47.0,40.530554,4.547689,-1.816404,127352.9400,-127352.9400,-85.3710,47.0,26554.972603,0.000214
1,12347,2011-08,1,12.830986,240.0,2.0,1822.0,142.0,20.640101,9.622845,...,142.0,3.547177,2.606898,32.738324,411.8400,8.3160,4648.8420,142.0,35.520500,8.907242
2,12348,2011-08,0,52.020833,144.0,1.0,2497.0,48.0,48.268526,0.931981,...,48.0,15.883893,3.721855,58.760625,396.0000,1.6500,2820.5100,48.0,67.342696,3.123453
3,12349,2011-08,0,9.233645,48.0,-1.0,988.0,107.0,8.408867,1.558634,...,107.0,50.462126,7.102276,40.818070,412.5000,-24.6675,4367.5335,107.0,54.673882,4.889357
4,12350,2011-08,0,11.588235,24.0,1.0,197.0,17.0,4.345383,0.472554,...,17.0,15.402339,4.098028,32.456471,66.0000,14.0250,551.7600,17.0,12.004638,1.178640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5203,18283,2011-08,0,1.592593,13.0,1.0,946.0,594.0,1.568776,5.021481,...,594.0,2.390382,3.172500,4.364722,36.1350,0.3465,2592.6450,594.0,3.767711,3.439766
5204,18284,2011-08,0,17.000000,50.0,-1.0,493.0,29.0,14.992855,1.006840,...,29.0,16.390112,4.144780,24.845586,82.5000,-41.2500,720.5220,29.0,20.064162,-0.050964
5205,18285,2011-08,0,12.083333,80.0,1.0,145.0,12.0,21.760926,3.258881,...,12.0,22.746537,2.931939,58.712500,336.6000,12.8700,704.5500,12.0,89.901391,3.158211
5206,18286,2011-08,0,8.457143,36.0,-8.0,592.0,70.0,7.224527,1.525244,...,70.0,11.243420,4.944969,28.012993,123.5025,-89.1000,1960.9095,70.0,26.685538,-0.669937


In [19]:
data.head()

,order_id,product_id,description,quantity,order_date,price,customer_id,country,total
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,11.4675,13085,United Kingdom,137.61
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,11.1375,13085,United Kingdom,133.65
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,11.1375,13085,United Kingdom,133.65
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,3.4650,13085,United Kingdom,166.32
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,2.0625,13085,United Kingdom,49.50


In [20]:
def get_high_correlation_cols(df, corrThresh=0.7):
    numeric_cols = df._get_numeric_data().columns
    corr_matrix = df.loc[:, numeric_cols].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > corrThresh)]
    return to_drop


In [21]:
get_high_correlation_cols(data)

             quantity     price  customer_id     total
quantity     1.000000  0.002557     0.007116  0.729222
price        0.002557  1.000000     0.003764  0.287810
customer_id  0.007116  0.003764     1.000000  0.005763
total        0.729222  0.287810     0.005763  1.000000


['total']